# Test Metrics.ipynb

Run all metrics on the test conversation.


In [5]:
# Load test_conversation.txt
with open('test_conversation.txt', 'r') as f:
    test_conversation = f.readlines()

# Print with each list item on a new line
for line in test_conversation:
    print(line)


Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?

Task: Generate 3 different possible one-step calculations to serve as step 1 in solving the problem. Only work on step 1. Put each calculation on a new line. Do not number them.

Add 16 and 4 and multiply by 2 

Subtract 3 from 16 and multiply by 2 

Multiply 16 and 2 and subtract 7

Task: State the calculation above that is most likely to contribute to solving the problem. If it fully solves the original problem, also output STOP and the solution to the problem. If none of the calculations are correct, output ERROR and generate three new ones.

Subtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56


In [28]:
# Load answer for this question

import json
import regex as re

# Load the test question from test.jsonl
with open('test.jsonl', 'r') as f:
    test = f.readlines()

# Strip items between << and >> in each line
test = [re.sub(r'<<.*?>>', '', line) for line in test]

#print(test)
print(json.loads(test[0])['answer'])


Janet sells 16 - 3 - 4 = 9 duck eggs a day.
She makes 9 * 2 = $18 every day at the farmer’s market.
#### 18


## Accuracy

Counts as zero points, model is incorrect.

## Length

It's important to use the OpenAI tiktoken tokenizer.

In [29]:
# Concatenate conversation

concat_convo = ''.join(test_conversation)

concat_convo


"Q: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\nTask: Generate 3 different possible one-step calculations to serve as step 1 in solving the problem. Only work on step 1. Put each calculation on a new line. Do not number them.\nAdd 16 and 4 and multiply by 2 \nSubtract 3 from 16 and multiply by 2 \nMultiply 16 and 2 and subtract 7\nTask: State the calculation above that is most likely to contribute to solving the problem. If it fully solves the original problem, also output STOP and the solution to the problem. If none of the calculations are correct, output ERROR and generate three new ones.\nSubtract 3 from 16 and multiply by 2 - STOP - 28 x 2 = $56"

In [30]:
import tiktoken

enc = tiktoken.encoding_for_model("text-davinci-003")

print(enc.encode("hello world"))
print(enc.encode(concat_convo))


[31373, 995]
[48, 25, 28111, 447, 247, 82, 39694, 3830, 1467, 9653, 583, 1110, 13, 1375, 25365, 1115, 329, 12607, 790, 3329, 290, 275, 1124, 27563, 1040, 329, 607, 2460, 790, 1110, 351, 1440, 13, 1375, 16015, 262, 17675, 379, 262, 9818, 6, 1910, 4445, 329, 720, 17, 583, 4713, 22045, 5935, 13, 1374, 881, 287, 5054, 857, 673, 787, 790, 1110, 379, 262, 9818, 6, 1910, 30, 198, 25714, 25, 2980, 378, 513, 1180, 1744, 530, 12, 9662, 16765, 284, 4691, 355, 2239, 352, 287, 18120, 262, 1917, 13, 5514, 670, 319, 2239, 352, 13, 5930, 1123, 17952, 319, 257, 649, 1627, 13, 2141, 407, 1271, 606, 13, 198, 4550, 1467, 290, 604, 290, 29162, 416, 362, 220, 198, 7004, 83, 974, 513, 422, 1467, 290, 29162, 416, 362, 220, 198, 15205, 541, 306, 1467, 290, 362, 290, 34128, 767, 198, 25714, 25, 1812, 262, 17952, 2029, 326, 318, 749, 1884, 284, 8676, 284, 18120, 262, 1917, 13, 1002, 340, 3938, 39107, 262, 2656, 1917, 11, 635, 5072, 44934, 290, 262, 4610, 284, 262, 1917, 13, 1002, 4844, 286, 262, 16765, 389, 3376

In [31]:
print(len(enc.encode(concat_convo)))


211


## Cost

2 cents per 1000 tokens

In [32]:
print("$" + str(len(enc.encode(concat_convo)) * (1 / 1000) * 0.02))


$0.00422


## Length Relative to Baseline

Going to add direct prompting baseline later, but for now just do relative to provided question and answer.

In [33]:
provided_question_answer = json.loads(test[0])['question'] + "\n" + json.loads(test[0])['answer']
print(provided_question_answer)

print('ratio')
print(len(enc.encode(concat_convo)) / len(enc.encode(provided_question_answer)))


Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Janet sells 16 - 3 - 4 = 9 duck eggs a day.
She makes 9 * 2 = $18 every day at the farmer’s market.
#### 18
2.0485436893203883


## Number of Reasoning Steps

Check linebreaks and periods.

In [34]:
print('num linebreaks')
print(concat_convo.count('\n'))

print('num periods')
print(concat_convo.count('.'))


num linebreaks
6
num periods
10


## Contrasting Words

Search for {'while', 'but', 'though', 'although', 'other', 'others', 'however'}.

In [35]:
print('num contrasting words')

for word in ['while', 'but', 'though', 'although', 'other', 'others', 'however']:
    print(word)
    print(concat_convo.count(word))


num contrasting words
while
0
but
1
though
0
although
0
other
0
others
0
however
0
